In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import datetime

# Define Sampling layer as a subclass of keras.layers.Layer
## Sampling layer: Layer that samples a random point in latent space from a distribution with a mean and variance

In [2]:

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Define latent space dimension

In [3]:
latent_dim = 2

# Encoder

In [4]:

encoder_inputs = keras.Input(shape=(6000,1))

x = layers.Conv1D(256,16,strides=2,padding='same',activation='relu',use_bias=True,kernel_initializer='VarianceScaling',bias_initializer = 'Zeros')(encoder_inputs)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

x = layers.Conv1D(128,8,strides=2,padding='same',activation='relu',use_bias=True,kernel_initializer='VarianceScaling',bias_initializer = 'Zeros')(x)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

x = layers.Conv1D(32,8,strides=2,padding='same',activation='relu',use_bias=True,kernel_initializer='VarianceScaling',bias_initializer = 'Zeros')(x)#possibly update kernel_initializer
#x = layers.MaxPooling1D(pool_size = 4,strides = 4, padding = 'same')(x)

shape_before_flattening = K.int_shape(x)

flatten_1 = layers.Flatten()(x)
x = layers.Dense(16 , activation="relu")(flatten_1)

z_mean = layers.Dense(latent_dim, name="z_mean",kernel_initializer='Zeros',bias_initializer = 'Zeros')(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var",kernel_initializer='Zeros',bias_initializer = 'Zeros')(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6000, 1)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 3000, 256)    4352        ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 1500, 128)    262272      ['conv1d[0][0]']                 
                                                                                                  
 conv1d_2 (Conv1D)              (None, 750, 32)      32800       ['conv1d_1[0][0]']               
                                                                                            

2022-03-15 19:48:58.720319: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-15 19:48:59.309901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22306 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


# Decoder

In [5]:

#DECODER
latent_inputs = keras.Input(shape=(latent_dim,))

# x = layers.Dense(1500, activation="relu")(latent_inputs)

#x = layers.Dense(1500, activation="relu")(latent_inputs)

x = layers.Dense(np.prod(shape_before_flattening[1:]), activation="relu")(latent_inputs)
x = layers.Reshape(shape_before_flattening[1:])(x)

x = layers.Conv1DTranspose(32, 8, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(128, 8, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(256, 16, activation="relu", strides=2, padding="same")(x)


decoder_outputs = layers.Conv1DTranspose(1, 16, padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 24000)             72000     
                                                                 
 reshape (Reshape)           (None, 750, 32)           0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 1500, 32)         8224      
 nspose)                                                         
                                                                 
 conv1d_transpose_1 (Conv1DT  (None, 3000, 128)        32896     
 ranspose)                                                       
                                                                 
 conv1d_transpose_2 (Conv1DT  (None, 6000, 256)        5245

# Defining subclass VAE
## VAE is a subclass of keras.Model class

In [13]:

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),axis=(1)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=0))
            total_loss = reconstruction_loss + kl_loss
            #total_loss = reconstruction_loss #ABSOLUTELY CHANGE!
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Load dataset
## And convert it to numpy array

In [7]:

train_X = pd.read_csv('/home/ege/Repo/SideChannel-AdversarialAI/Tensorflow/DataSet/trainX13.csv', header=None)
train_Y = pd.read_csv('/home/ege/Repo/SideChannel-AdversarialAI/Tensorflow/DataSet/trainY13.csv', header=None)

trainY = train_Y.to_numpy()
trainX = train_X.to_numpy()
trainX = np.expand_dims(trainX,axis=2)

# Normalize dataset

In [8]:
minimum = np.amin(trainX)
maximum = np.amax(trainX)

trainX = (trainX-minimum)/(maximum-minimum)

# Train model

In [14]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history = vae.fit(trainX, epochs=50, batch_size=32)

Epoch 1/50
53/53 [==============================] - 3s 33ms/step - loss: 2216.9816 - reconstruction_loss: 2200.0488 - kl_loss: 0.0040
Epoch 2/50
53/53 [==============================] - 2s 33ms/step - loss: 2193.4949 - reconstruction_loss: 2198.4072 - kl_loss: 7.1405e-04
Epoch 3/50
53/53 [==============================] - 2s 33ms/step - loss: 2169.7324 - reconstruction_loss: 2190.6934 - kl_loss: 1.6812e-04
Epoch 4/50
53/53 [==============================] - 2s 33ms/step - loss: 2178.6730 - reconstruction_loss: 2191.8064 - kl_loss: 1.4971e-04
Epoch 5/50
53/53 [==============================] - 2s 33ms/step - loss: 2263.1919 - reconstruction_loss: 2191.9373 - kl_loss: 3.5633e-04
Epoch 6/50
53/53 [==============================] - 2s 33ms/step - loss: 2197.6132 - reconstruction_loss: 2193.1692 - kl_loss: 1.8481e-04
Epoch 7/50
53/53 [==============================] - 2s 33ms/step - loss: 2185.0271 - reconstruction_loss: 2190.3062 - kl_loss: 8.3532e-05
Epoch 8/50
53/53 [====================

# Load classification model

In [10]:
classification_model = tf.keras.models.load_model('TrainedModel/trainedModel.h5')

# Test reconstructed dataset with the classification model

In [11]:

for i in range(14):
    

    trainXCUT = trainX[i::14]
    trainYCUT = trainY[i::14]

    z_mean, z_log_var, z = vae.encoder.predict(trainXCUT)
    reconstructed_x = vae.decoder.predict(z)*(maximum-minimum)+minimum

    # if(i == 5):
    #     sampleToPredict = 0
    #     (reconstructed_x[sampleToPredict]).tofile('prediction.csv', sep = ',')
    #     ((trainXCUT[sampleToPredict].T)*(maximum-minimum)+minimum).tofile('predictionTARGET.csv', sep = ',')

    result = classification_model.evaluate((reconstructed_x),trainYCUT)

4/4 [==============================] - 0s 4ms/step - loss: 11.5878 - accuracy: 0.0000e+00


In [12]:
print(reconstructed_x)

[[[7.4486394]
  [5.932315 ]
  [5.7359467]
  ...
  [3.8509004]
  [3.786337 ]
  [3.6813483]]

 [[7.1111174]
  [5.7287173]
  [5.564599 ]
  ...
  [3.7635245]
  [3.6840127]
  [3.602539 ]]

 [[7.2215276]
  [5.8008356]
  [5.595108 ]
  ...
  [3.7892792]
  [3.6995034]
  [3.6251626]]

 ...

 [[7.179163 ]
  [5.77273  ]
  [5.6082335]
  ...
  [3.7901685]
  [3.7056594]
  [3.6240547]]

 [[7.347051 ]
  [5.877773 ]
  [5.708166 ]
  ...
  [3.805622 ]
  [3.729754 ]
  [3.6409411]]

 [[7.292639 ]
  [5.842793 ]
  [5.6771965]
  ...
  [3.7773445]
  [3.7090282]
  [3.6171415]]]
